In [4]:
import pandas as pd

In [5]:
movies = pd.read_csv("../../data/external/MovieLensDataSets/movies.csv")
ratings = pd.read_csv("../../data/external/MovieLensDataSets/ratings.csv")
tags = pd.read_csv("../../data/external/MovieLensDataSets/tags.csv")
links = pd.read_csv("../../data/external/MovieLensDataSets/links.csv")

In [7]:
tags_agregadas = tags.groupby('movieId')['tag'].apply(lambda x: ' | '.join(x.dropna().astype(str))).reset_index()

info_filmes = pd.merge(movies, links, on='movieId', how='left')
info_filmes_completa = pd.merge(info_filmes, tags_agregadas, on='movieId', how='left')

df_final = pd.merge(info_filmes_completa, ratings, on='movieId', how='inner')

df_final['rating'] = pd.to_numeric(df_final['rating'])
df_final['timestamp'] = pd.to_datetime(df_final['timestamp'], unit='s')


Primeiras 5 linhas e ultimas 5 linhas e informações dos dados

In [17]:
df_final. head()

,movieId,title,genres,imdbId,tmdbId,tag,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,children | Disney | animation | children | Dis...,10,2.5,2007-01-20 03:53:51
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,children | Disney | animation | children | Dis...,11,3.0,1996-12-08 22:44:36
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,children | Disney | animation | children | Dis...,17,4.0,2002-07-22 02:42:31
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,children | Disney | animation | children | Dis...,19,3.0,2000-11-20 07:14:48
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,children | Disney | animation | children | Dis...,20,5.0,2019-03-21 16:03:50


In [18]:
df_final.tail()

,movieId,title,genres,imdbId,tmdbId,tag,userId,rating,timestamp
32000199,292731,The Monroy Affaire (2022),Drama,26812510,1032473.0,NaN,50685,4.0,2023-10-12 01:39:39
32000200,292737,Shelter in Solitude (2023),Comedy|Drama,14907358,986674.0,NaN,165364,1.5,2023-10-12 21:25:47
32000201,292753,Orca (2023),Drama,12388280,948139.0,NaN,150812,4.0,2023-10-12 19:55:30
32000202,292755,The Angry Breed (1968),Drama,64027,182776.0,NaN,91097,1.0,2023-10-12 21:08:48
32000203,292757,Race to the Summit (2023),Action|Adventure|Documentary,28995566,1174725.0,NaN,171418,3.5,2023-10-12 22:21:28


In [19]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000204 entries, 0 to 32000203
Data columns (total 9 columns):
 #   Column     Dtype         
---  ------     -----         
 0   movieId    int64         
 1   title      object        
 2   genres     object        
 3   imdbId     int64         
 4   tmdbId     float64       
 5   tag        object        
 6   userId     int64         
 7   rating     float64       
 8   timestamp  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 2.1+ GB


In [8]:
# 1. Verificando valores nulos no DataFrame final
print("--- Contagem de Nulos no df_final ---")
print(df_final.isnull().sum())

# 2. Verificando linhas duplicadas
# (Ex: um usuário avaliou o mesmo filme duas vezes? Isso pode acontecer)
duplicatas = df_final.duplicated().sum()
print(f"\nTotal de linhas duplicadas: {duplicatas}")

# 3. (Opcional) Removendo duplicatas, se existirem
if duplicatas > 0:
    df_final = df_final.drop_duplicates()
    print("Linhas duplicadas foram removidas.")

# 4. Engenharia de Features a partir do 'timestamp'
# Isso é MUITO útil para a análise. Já convertemos para datetime.
# Vamos extrair o Ano, Mês e Dia da Semana da avaliação.
df_final['ano_avaliacao'] = df_final['timestamp'].dt.year
df_final['mes_avaliacao'] = df_final['timestamp'].dt.month
df_final['dia_semana_avaliacao'] = df_final['timestamp'].dt.day_name()

print("\n--- DataFrame com Novas Colunas de Tempo ---")
print(df_final[['timestamp', 'ano_avaliacao', 'mes_avaliacao', 'dia_semana_avaliacao']].head())

--- Contagem de Nulos no df_final ---
movieId           0
title             0
genres            0
imdbId            0
tmdbId         4726
tag          175367
userId            0
rating            0
timestamp         0
dtype: int64

Total de linhas duplicadas: 0

--- DataFrame com Novas Colunas de Tempo ---
            timestamp  ano_avaliacao  mes_avaliacao dia_semana_avaliacao
0 2007-01-20 03:53:51           2007              1             Saturday
1 1996-12-08 22:44:36           1996             12               Sunday
2 2002-07-22 02:42:31           2002              7               Monday
3 2000-11-20 07:14:48           2000             11               Monday
4 2019-03-21 16:03:50           2019              3             Thursday
